In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# data = pd.read_csv(".csv")
# data.head()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
benign=pd.read_csv('/content/drive/MyDrive/iot-dataset/2/2.benign.csv')
g_c=pd.read_csv('/content/drive/MyDrive/iot-dataset/2/2.gafgyt.combo.csv')
g_j=pd.read_csv('/content/drive/MyDrive/iot-dataset/2/2.gafgyt.junk.csv')
g_s=pd.read_csv('/content/drive/MyDrive/iot-dataset/2/2.gafgyt.scan.csv')
g_t=pd.read_csv('/content/drive/MyDrive/iot-dataset/2/2.gafgyt.tcp.csv')
g_u=pd.read_csv('/content/drive/MyDrive/iot-dataset/2/2.gafgyt.udp.csv')
m_a=pd.read_csv('/content/drive/MyDrive/iot-dataset/2/2.mirai.ack.csv')
m_sc=pd.read_csv('/content/drive/MyDrive/iot-dataset/2/2.mirai.scan.csv')
m_sy=pd.read_csv('/content/drive/MyDrive/iot-dataset/2/2.mirai.syn.csv')
m_u=pd.read_csv('/content/drive/MyDrive/iot-dataset/2/2.mirai.udp.csv')
m_u_p=pd.read_csv('/content/drive/MyDrive/iot-dataset/2/2.mirai.udpplain.csv')

In [ ]:
benign['type']='benign'
m_u['type']='attack'
g_c['type']='attack'
g_j['type']='attack'
g_s['type']='attack'
g_t['type']='attack'
g_u['type']='attack'
m_a['type']='attack'
m_sc['type']='attack'
m_sy['type']='attack'
m_u_p['type']='attack'

In [ ]:
data=pd.concat([benign,m_u,g_c,g_j,g_s,g_t,g_u,m_a,m_sc,m_sy,m_u_p],
               axis=0, sort=False, ignore_index=True)

In [ ]:
data.head()

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc,type
0,1.000000,60.000000,0.000000,1.000000,60.000000,0.000000,1.000000,60.000000,0.000000,1.000000,...,0.0,0.0,1.000000,60.000000,0.000000,60.000000,0.000000,0.0,0.0,benign
1,1.000000,110.000000,0.000000,1.000000,110.000000,0.000000,1.000000,110.000000,0.000000,1.000000,...,0.0,0.0,1.238636,106.146795,7.887770,106.146795,62.216910,0.0,0.0,benign
2,1.724055,91.439104,249.441812,1.823878,92.454965,253.612868,1.937463,93.483558,255.733288,1.993563,...,0.0,0.0,1.000000,78.000000,0.000000,78.000000,0.000000,0.0,0.0,benign
3,1.000000,342.000000,0.000000,1.000000,342.000000,0.000000,1.000000,342.000000,0.000000,1.000000,...,0.0,0.0,2.487473,341.959189,7.347919,341.959189,53.991908,0.0,0.0,benign
4,2.516815,90.867308,150.827639,2.688981,91.541993,160.705060,2.888469,92.277536,169.944112,2.988463,...,0.0,0.0,2.237519,98.930412,9.942635,98.930412,98.855983,0.0,0.0,benign


In [ ]:
x = data.iloc[:,:-1].values
y = data.iloc[:,-1].values

In [ ]:
#standardize numerical columns
# def standardize(df,col):
#     df[col]= (df[col]-df[col].mean())/df[col].std()
# data_st=data.copy()
# for i in (data_st.iloc[:,:-1].columns):
#     standardize (data_st,i)
# data_st.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x = sc.fit_transform(x)

In [ ]:
x = x.reshape(x.shape[0], x.shape[1], 1)
print(x.shape)

(835876, 115, 1)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2,shuffle=True)

In [ ]:
from keras import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D

model = Sequential()
model.add(Conv1D(64, 2, activation="relu", batch_input_shape=(None, 115, 1)))
model.add(Dense(16, activation="relu"))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(2, activation = 'softmax'))
model.compile(loss = 'sparse_categorical_crossentropy', 
     optimizer = "adam",               
              metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 114, 64)           192       
                                                                 
 dense_2 (Dense)             (None, 114, 16)           1040      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 57, 16)           0         
 1D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 912)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 1826      
                                                                 
Total params: 3,058
Trainable params: 3,058
Non-trainable params: 0
____________________________________________________

In [ ]:
#Fitting the data to the training dataset
model.fit(X_train,y_train, validation_data=(X_test,y_test), epochs=1)

20897/20897 [==============================] - 170s 8ms/step - loss: 0.0022 - accuracy: 0.9996 - val_loss: 9.6561e-04 - val_accuracy: 0.9998


In [ ]:
y_pred=model.predict(X_test)
y_pred = y_pred.argmax(axis=-1)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, matthews_corrcoef, f1_score, precision_score, recall_score

print(f"Accuracy   : {accuracy_score(y_test, y_pred)}")
print(f"Precision  : {precision_score(y_test, y_pred)}")
print(f"F1-Measure : {f1_score(y_test, y_pred)}")
print(f"MCC        : {matthews_corrcoef(y_test, y_pred)}")
print(f"Recall     : {recall_score(y_test, y_pred)}")

Accuracy   : 0.9998325118438053
Precision  : 1.0
F1-Measure : 0.9945987654320987
MCC        : 0.9945286724932503
Recall     : 0.9892555640828856


In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    164570
           1       1.00      0.99      0.99      2606

    accuracy                           1.00    167176
   macro avg       1.00      0.99      1.00    167176
weighted avg       1.00      1.00      1.00    167176



In [ ]:
print(confusion_matrix(y_test, y_pred))

[[164570      0]
 [    28   2578]]
